In [1]:
import json_lines
import re
import json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
import string
import pandas as pd
import mtranslate

In [2]:
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [3]:
df = pd.DataFrame(columns=['Review', 'Topic', 'Sentiment', 'Source'])
lemmatizer = WordNetLemmatizer()

#For some reason, please returns positive sentiment because it assumes 'pleasing' but normally people just asking politely
weird_sentiment_words = ['la', 'please']

#file = json.loads(open('chi.json', encoding="utf8"))
with open('fitnessfirst.json', 'rb') as f:
    for line in json_lines.reader(f):

        try:
            #Check if key exists
            sentence = line['reviewBody']
            #print(line['message'])

            print("New review")
            print('---------------------------------------')

            #Translate to English
            sentence = mtranslate.translate(sentence, 'en', 'auto')
            
            #Split a single comment into multiple sentences
            sentences = sent_tokenize(sentence)
            #print(sentences)

            total_sentiment = 0
            list_of_nouns = []
            
            #Sentiment for each sentence
            for s in sentences:

                s = s.lower()
                s = s.replace('fitness first', ' ')
                s = re.sub(r"\byin\b", "", s)
                s = re.sub(r"\byang\b", "", s)

                print("Sentence: ", s)
                sentiment = 0.0
                #print(s)

                tokenized = word_tokenize(s)
                #print(tokenized)
                
                new_tokenized = [x for x in tokenized if x not in weird_sentiment_words]
                tokenized = new_tokenized

                #POS tag each word
                tagged = pos_tag(tokenized)
                print(tagged)

                previous_word = ""
                

                #Get the sentiment for each word
                for w, t in tagged:
                    
                    sentiment_added = False
                    
                    #Convert Penn Treebank POS tag to Wordnet POS tag
                    wn_tag = penn_to_wn(t)
                    
                    #If successfully converted, lemmatize using the Wordnet POS tag
                    #If not successfully converted, lemmatize without any POS tags
                    if wn_tag is None:
                        lemma = lemmatizer.lemmatize(w)
                    else:
                        lemma = lemmatizer.lemmatize(w, pos=wn_tag)
                        
                    #If can't lemmatize, set the word to be passed into SentiWordnet as the initial word, w
                    #If can lemmatize, set the word to be passed into Sentiwordnet as the lemmatized word, lemma
                    if not lemma:
                        print("Can't lemmatize: ", w)
                        final_word = w
                    else:
                        print("Lemmatized word: ", lemma)
                        final_word = lemma
                        
                    #Make cool and wow positive words
                    if (final_word in ['cool', 'wow', 'glad']) and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['cool', 'wow', 'glad']):
                        print(final_word, " has a score of 0.5")
                        sentiment = sentiment + 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice') and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice'):
                        print(final_word, " has a score of 0.875")
                        sentiment = sentiment + 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel']) and (previous_word == 'not'):
                        sentiment = sentiment + 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel']):
                        sentiment = sentiment - 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                        
                    #If the Wordnet POS tag exists, use it to get the synset of the word
                    if wn_tag is None:
                        synsets = wn.synsets(final_word)
                    else:
                        synsets = wn.synsets(final_word, pos=wn_tag)
                        
                    #If no synsets found, skip the word
                    if not synsets:
                        print("Synset not found")
                        continue
                        
                    # Take the first sense of the word which is the most common
                    synset = synsets[0]
                    swn_synset = swn.senti_synset(synset.name())
                    
                    if sentiment_added == False:
                        if final_word != 'fitness' and previous_word == 'not':      
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment - swn_synset.pos_score() - swn_synset.neg_score()
                        elif final_word != 'fitness':
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment + swn_synset.pos_score() - swn_synset.neg_score()
                        else:
                            print("The word fitness is not considered as a sentiment")
                    
                    previous_word = final_word

                    if t.startswith('N') or w == 'post':
                        if w != 'i' and w != 'la' and w != 'hi':
                            list_of_nouns.append(w)       
                            
                # sum greater than 0 => positive sentiment
                total_sentiment += sentiment

                    
            df2 = pd.DataFrame([[sentence, ','.join(list_of_nouns), total_sentiment, 'Facebook Review Fitness First']],columns=['Review', 'Topic', 'Sentiment', 'Source'])

            df = pd.concat([df2,df], ignore_index=True)
        except:
            continue

New review
---------------------------------------
Sentence:  best.
[('best', 'JJS'), ('.', '.')]
Lemmatized word:  best
best  |  Synset('best.a.01')  |  <best.a.01: PosScore=0.75 NegScore=0.0>  |  (superlative of `good') having the most positive qualities
Lemmatized word:  .
Synset not found
New review
---------------------------------------
Sentence:   's management service getting from bad to worst for my 15 years journey with them especially i am terminating my membership now.
[("'s", 'POS'), ('management', 'NN'), ('service', 'NN'), ('getting', 'VBG'), ('from', 'IN'), ('bad', 'JJ'), ('to', 'TO'), ('worst', 'VB'), ('for', 'IN'), ('my', 'PRP$'), ('15', 'CD'), ('years', 'NNS'), ('journey', 'NN'), ('with', 'IN'), ('them', 'PRP'), ('especially', 'RB'), ('i', 'VBP'), ('am', 'VBP'), ('terminating', 'VBG'), ('my', 'PRP$'), ('membership', 'NN'), ('now', 'RB'), ('.', '.')]
Lemmatized word:  's
Synset not found
Lemmatized word:  management
management  |  Synset('management.n.01')  |  <managem

In [4]:
#df.to_json('chi_fitness_comments_sentiment.json')

In [5]:
df_filtered = df[df['Sentiment'] != 0]
print(len(df_filtered[df_filtered['Sentiment'] < 0]))
print(len(df_filtered[df_filtered['Sentiment'] > 0]))

15
4


In [6]:
df_filtered[df_filtered['Sentiment'] > 0]

,Review,Topic,Sentiment,Source
0,I’ve been a member of FF for many years. Of la...,"member,years,standards,home,gym,empire,water,d...",1.625,Facebook Review Fitness First
3,Setia City mall class schedule in app and webs...,"city,mall,class,schedule,pilates,class,class,b...",1.375,Facebook Review Fitness First
13,I have been informed very friendly by your fro...,"front,desk,mall,sign,members,members,front,des...",2.750,Facebook Review Fitness First
19,Best.,,0.750,Facebook Review Fitness First


In [7]:
df_filtered[df_filtered['Sentiment'] < 0]

,Review,Topic,Sentiment,Source
1,Poor services and i had been cheated by u! As...,"services,u,rate,system,charge,rate,staff,excus...",-1.625,Facebook Review Fitness First
2,Hi I have been a fitness first member for more...,"member,years,classes,paradigm,empire,year,empi...",-0.750,Facebook Review Fitness First
4,I think FF members like to roll in the sweat o...,"members,sweat,way,equipment,use,towel,body,mac...",-0.625,Facebook Review Fitness First
5,Relocation of FF IOI Mall has made me and many...,"relocation,mall,way,management,things,yoga,cla...",-0.125,Facebook Review Fitness First
6,Poor service. Could not even provide basic nec...,"service,necessity,hand,towel,morning,avenue,in...",-1.750,Facebook Review Fitness First
8,"Within 3 months plus I joined the FF, the staf...","months,staff,membership,card,time,curve,branch...",-2.625,Facebook Review Fitness First
9,"I have always prefer FF than other clubs, prob...","clubs,platinum,member,platinum,clubs,club,axis...",-0.750,Facebook Review Fitness First
10,The facilities provided are not up to the stan...,"facilities,standard,amount,fee,month,empire,wa...",-3.500,Facebook Review Fitness First
11,wish I read all the recent reviews here before...,"reviews,experience,service,membership,word,car...",-1.750,Facebook Review Fitness First
12,I cancelled my membership since May 2017. I wa...,"membership,curve,branch,form,cancellation,emai...",-1.375,Facebook Review Fitness First


In [8]:
df_filtered.to_json('fitnessfirstreviewsentiment.json', orient='records')